In [1]:
%pip install tensorflow_datasets
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import KFold

2024-05-13 06:22:38.355718: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 06:22:38.358489: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 06:22:38.395661: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 06:22:39.350249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "../../dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=1000, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "../../models/raw_data/training_batchSize30_epoch1000.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.3f}')
else:
    print("No folds to calculate average accuracy.")

Found 240 files belonging to 60 classes.


Training for fold 1
Train Index [  0   1   2   3   4   6   7   8  10  11  12  13  14  17  20  21  22  23
  25  26  27  28  29  31  32  33  34  35  36  37  38  39  40  41  42  43
  44  46  47  48  49  50  51  52  53  54  55  56  57  58  59  61  62  63
  64  70  71  72  73  74  75  77  79  80  81  82  83  84  85  86  87  88
  89  91  92  93  94  95  96  97  99 100 101 102 103 104 105 106 107 108
 109 110 116 117 119 120 121 122 123 124 125 126 127 128 129 130 131 132
 134 135 137 138 139 140 144 146 147 148 149 150 151 152 155 156 157 159
 160 161 162 163 164 167 168 169 172 173 174 175 176 178 179 180 181 182
 183 184 185 186 187 188 189 190 191]
Test Index [  5   9  15  16  18  19  24  30  45  60  65  66  67  68  69  76  78  90
  98 111 112 113 114 115 118 133 136 141 142 143 145 153 154 158 165 166
 170 171 177]


2024-05-13 06:35:22.405011: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 387ms/step - accuracy: 0.0028 - loss: 0.7940 - val_accuracy: 0.0769 - val_loss: 0.1279
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.1758 - loss: 0.1355 - val_accuracy: 0.1795 - val_loss: 0.1334
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.2227 - loss: 0.1206 - val_accuracy: 0.2564 - val_loss: 0.1049
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.4210 - loss: 0.0813 - val_accuracy: 0.3333 - val_loss: 0.0777
Epoch 5/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.6334 - loss: 0.0483 - val_accuracy: 0.4359 - val_loss: 0.0636
Epoch 6/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.7392 - loss: 0.0307 - val_accuracy: 0.5385 - val_loss: 0.0533
Epoch 7/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.8958 - loss: 0.0209 - val_accuracy: 0.5897 - val_loss: 0.0447
Epoch 8/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.9358 - loss: 0.0151 - val_accuracy: 0.

Predicted Classes: [38 42 22 48 18  4 27 41 43  5 29 58 17  5 17 45 18  3 13 31 43 50  0 34
  4 11  5 20 42 19 40 37 31 37 22 57 29 56 34 48 37 56 14 37 52 21 50 12]
True Classes: [38 42 22 48 18  4 19 41 43  8 29 58 17 28 17 45 18  3 13 31 43 50  0 28
  4 11  5 20 42 19 40 36 31 12 22 57 29 28 28 48 36 56 14 37 52 21 50 12]
Final model saved to ../../models/raw_data/training_batchSize30_epoch1000.h5
Accuracy for fold 1: 0.8125
Accuracy for fold 2: 0.7291666666666666
Accuracy for fold 3: 0.8333333333333334
Accuracy for fold 4: 0.875
Accuracy for fold 5: 0.8125
Average Accuracy: 0.812


Penambahan DropOut

In [5]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "../../dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)  # Adding dropout layer with 50% dropout rate
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)
    print ('Train Index', train_index)
    print ('Test Index', test_index)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=1000, validation_data=(X_val_fold, Y_val_fold))

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Save the model after all folds are processed
model_save_path = "../../models/raw_data/training_withDropOut_batchSize30_epoch1000.h5"
model.save(model_save_path)
print(f"Final model saved to {model_save_path}")

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

# Print Average Accuracy 
total_acc = sum(acc_per_fold)  # Summing up all accuracies
num_folds = len(acc_per_fold)  # Getting the number of folds

if num_folds > 0:
    average_acc = total_acc / num_folds  # Calculating average accuracy
    print(f'Average Accuracy: {average_acc:.3f}')
else:
    print("No folds to calculate average accuracy.")

Found 240 files belonging to 60 classes.
Training for fold 1
Train Index [  0   1   2   3   4   6   7   8  10  11  12  13  14  17  20  21  22  23
  25  26  27  28  29  31  32  33  34  35  36  37  38  39  40  41  42  43
  44  46  47  48  49  50  51  52  53  54  55  56  57  58  59  61  62  63
  64  70  71  72  73  74  75  77  79  80  81  82  83  84  85  86  87  88
  89  91  92  93  94  95  96  97  99 100 101 102 103 104 105 106 107 108
 109 110 116 117 119 120 121 122 123 124 125 126 127 128 129 130 131 132
 134 135 137 138 139 140 144 146 147 148 149 150 151 152 155 156 157 159
 160 161 162 163 164 167 168 169 172 173 174 175 176 178 179 180 181 182
 183 184 185 186 187 188 189 190 191]
Test Index [  5   9  15  16  18  19  24  30  45  60  65  66  67  68  69  76  78  90
  98 111 112 113 114 115 118 133 136 141 142 143 145 153 154 158 165 166
 170 171 177]


2024-05-13 08:02:44.208582: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 382ms/step - accuracy: 0.0339 - loss: 1.1651 - val_accuracy: 0.0000e+00 - val_loss: 0.2788
Epoch 2/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.0108 - loss: 0.5633 - val_accuracy: 0.0256 - val_loss: 0.2913
Epoch 3/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.0335 - loss: 0.4477 - val_accuracy: 0.0256 - val_loss: 0.2079
Epoch 4/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.0403 - loss: 0.3901 - val_accuracy: 0.1026 - val_loss: 0.1394
Epoch 5/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.0291 - loss: 0.3468 - val_accuracy: 0.1282 - val_loss: 0.1164
Epoch 6/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.0404 - loss: 0.2855 - val_accuracy: 0.0513 - val_loss: 0.1123
Epoch 7/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.0276 - loss: 0.2681 - val_accuracy: 0.1026 - val_loss: 0.0971
Epoch 8/1000
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.0623 - loss: 0.2443 - val_accuracy

Predicted Classes: [38 42  2 48 18  4 27 41 43  8 58 58 17 34 34 45 18  3 13 31 43 20  0 34
  4 11  5 20 42 19 40 36 40 12 14 57 29 34 34 48 37 56 14 37 52 21 50 12]
True Classes: [38 42 22 48 18  4 19 41 43  8 29 58 17 28 17 45 18  3 13 31 43 50  0 28
  4 11  5 20 42 19 40 36 31 12 22 57 29 28 28 48 36 56 14 37 52 21 50 12]
Final model saved to ../../models/raw_data/training_withDropOut_batchSize30_epoch1000.h5
Accuracy for fold 1: 0.7083333333333334
Accuracy for fold 2: 0.3333333333333333
Accuracy for fold 3: 0.5625
Accuracy for fold 4: 0.75
Accuracy for fold 5: 0.75
Average Accuracy: 0.621
